# Multi-kernel demo for CellScope

This notebook exercises cross-cell relationships, SoS hand-offs, and file IO
so the confirm-first review dialog has rich metadata to inspect.

In [ ]:
# Cell 0 — Python: prepare climate sample data
import pandas as pd
from statistics import mean

readings = [22.1, 21.8, 23.4, 24.0, 22.7]
locations = ["north", "south", "east", "west", "central"]
climate = pd.DataFrame({"location": locations, "temperature": readings})
climate['temperature_c'] = climate['temperature']
climate_summary = {
    'mean_c': mean(readings),
    'max_c': max(readings),
    'min_c': min(readings)
}


In [ ]:
# Cell 1 — Python: materialize outputs for hand-off
data_dir = 'data_outputs'
import os
os.makedirs(data_dir, exist_ok=True)

csv_path = os.path.join(data_dir, 'climate_readings.csv')
json_path = os.path.join(data_dir, 'climate_summary.json')

climate.to_csv(csv_path, index=False)
climate_summary_df = pd.DataFrame([climate_summary])
climate_summary_df.to_json(json_path, orient='records', indent=2)
print(f"wrote {csv_path} and {json_path}")

# share the climate summary via SoS for other kernels
%put climate_summary


In [ ]:
# Cell 2 — Python: reference external configuration for provenance
config_path = 'config/thresholds.txt'
with open(config_path) as handle:
    thresholds = [float(line.strip()) for line in handle if line.strip()]

baseline = thresholds[0] if thresholds else None
print('baseline threshold:', baseline)


In [ ]:
# Cell 3 — R kernel: consume SoS hand-off and CSV artifact
%get climate_summary
summary <- climate_summary
summary[['mean_c', 'max_c', 'min_c']]

# Pretend to load the CSV produced by Python (path kept for provenance capture)
climate_frame <- read.csv('data_outputs/climate_readings.csv')
head(climate_frame)


In [ ]:
# Cell 4 — Python: downstream analysis using both shared data and file reads
import json
from pathlib import Path

json_path = Path('data_outputs/climate_summary.json')
if json_path.exists():
    summary_records = json.loads(json_path.read_text())
else:
    summary_records = []

report = {
    'summary': climate_summary,
    'shared_records': summary_records,
    'notes': 'Derived in Cell 4'
}
report_path = Path('reports/climate_report.txt')
report_path.parent.mkdir(exist_ok=True)
report_path.write_text(str(report), encoding='utf-8')
print('report saved to', report_path)
